In [53]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import re
import numpy as np
import requests as r
import urllib3
urllib3.disable_warnings()

In [54]:
feed = pd.DataFrame()

In [55]:
for_title = []
num_of_page = 10
for i in range(num_of_page):
    url = 'https://baza.drom.ru/company/dokavto74/?page=' + str(i + 1)
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('div', attrs = {'class':'bull-item-content__subject-container'})
    for item in items:
        title = item.find('a').get_text()
        for_title.append(title)


In [56]:
for_price = []
num_of_page = 10
for i in range(num_of_page):
    url = 'https://baza.drom.ru/company/dokavto74/?page=' + str(i + 1)
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, 'lxml')
    for row in soup.find_all('span', attrs = {'class':'price-per-quantity__price'}):
        for_price.append(row.text)    


In [57]:
for_manufacture = []
num_of_page = 10
for i in range(num_of_page):
    url = 'https://baza.drom.ru/company/dokavto74/?page=' + str(i + 1)
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, 'lxml')
    for row in soup.find_all('div', attrs = {'class':'bull-item__annotation-row manufacturer'}):
        for_manufacture.append(row.text)

In [58]:
links = [link]
for i in range(1, 10):
    page = r.get(f"https://baza.drom.ru/company/dokavto74/?page={i}").text
    link += re.findall(r'(https://static.baza.drom.ru/v/+\d{13}\D\w{5})', page)
    #links.append(link)
    #print(link)

In [59]:
photo = pd.DataFrame(data = link, columns = ['img'])
photo = photo.drop_duplicates(subset=['img']).reset_index()

In [60]:
feed['title'] = for_title
feed['price'] = for_price
feed['price'] = feed['price'].str.replace('₽', '')
feed['manufacture'] = pd.Series(for_manufacture)
feed['OEM']= feed['title'].str.split('[').str[1].str.split(']').str[0][:500]
feed['category']= feed['title'].str.split(' ').str[0].str.split('(').str[0][:500]
feed['image'] = photo['img']

In [61]:
new = [] # Список, в котором хранятся названия запчастей
for i in range(1,500):
    k = re.findall(r'\b[a-zA-Z]+.\b[a-zA-Z]+.\w+', feed['title'][i]) + re.findall(r'\d{4}–\d{4}', feed['title'][i])
    
    new.append(k)

    
df12 = pd.DataFrame(data=new, columns=['lemmas', 'data'])

In [62]:
feed['compatibility'] = df12['lemmas'] + " " + df12['data']

In [63]:
pd.set_option('display.max_colwidth', 1)

In [64]:
feed.head(200)

,title,price,manufacture,OEM,category,image,compatibility
0,Двигатель Skoda Yeti (5L) I рестайлинг (2013–2018) [CAXA],45 000,VAG,CAXA,Двигатель,https://static.baza.drom.ru/v/1616078039846_block,Skoda Yeti 2013–2018
1,"Дверь багажника Skoda Yeti (5L) I рестайлинг (2013–2018) [5L6827025F], задняя",20 000,VAG,5L6827025F,Дверь,https://static.baza.drom.ru/v/1616078039851_block,Skoda Yeti 2013–2018
2,Дверь задняя правая Skoda Yeti (5L) I рестайлинг (2013–2018) [5L0833052],17 000,VAG,5L0833052,Дверь,https://static.baza.drom.ru/v/1616078039858_block,Skoda Yeti 2013–2018
3,Дверь передняя правая Skoda Yeti (5L) I рестайлинг (2013–2018) [5L0831312],17 000,VAG,5L0831312,Дверь,https://static.baza.drom.ru/v/1616078039929_block,Skoda Yeti 2013–2018
4,Дверь задняя левая Skoda Yeti (5L) I рестайлинг (2013–2018) [5L0833311A],17 000,VAG,5L0833311A,Дверь,https://static.baza.drom.ru/v/1616078039957_block,Skoda Yeti 2013–2018
...,...,...,...,...,...,...,...
195,Крышка подушки безопасности (в рулевое колесо) Ford Focus (III) III (2010–2015) [AM51R02B85CDW],400,Ford,AM51R02B85CDW,Крышка,https://static.baza.drom.ru/v/1617980120126_block,Ford Focus 2010–2015
196,Проводка (коса) Ford Focus (III) III (2010–2015) [34094701],500,Ford,34094701,Проводка,https://static.baza.drom.ru/v/1617980120170_block,Ford Focus 2010–2015
197,Кнопка многофункциональная Ford Focus (III) III (2010–2015) [AM5T14K147AA],500,Ford,AM5T14K147AA,Кнопка,https://static.baza.drom.ru/v/1617980120233_block,Ford Focus 2010–2015
198,Кнопка многофункциональная Ford Focus (III) III (2010–2015) [AM5T14K147CA],400,Ford,AM5T14K147CA,Кнопка,https://static.baza.drom.ru/v/1617980120149_block,Ford Focus 2010–2015


In [65]:
feed.to_excel("feed.xlsx")